In [2]:
#Set up the environment
!pip install -q transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer
import numpy as np
import torch
from __future__ import annotations
from typing import Optional
from torch import Tensor
from torch.nn import Module
from transformers import BatchEncoding, PreTrainedTokenizerBase

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#Mount the google drvie
path = '/content/drive/My Drive/Sentiment Analysis/data.csv'
dataset = pd.read_csv(path)

In [6]:
# Split the dataset into training and validation sets
train_df, val_df = train_test_split(dataset, test_size=0.2, random_state=42)

# Create Dataset objects
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Create DatasetDict object
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})

dataset = dataset.remove_columns("__index_level_0__")
dataset = dataset.remove_columns("Unnamed: 0")
# Display the dataset
dataset

DatasetDict({
    train: Dataset({
        features: ['ID', 'Tweet', 'Optimistic', 'Thankful', 'Empathetic', 'Pessimistic', 'Anxious', 'Sad', 'Annoyed', 'Denial', 'Official report', 'Surprise', 'Joking'],
        num_rows: 8000
    })
    validation: Dataset({
        features: ['ID', 'Tweet', 'Optimistic', 'Thankful', 'Empathetic', 'Pessimistic', 'Anxious', 'Sad', 'Annoyed', 'Denial', 'Official report', 'Surprise', 'Joking'],
        num_rows: 2000
    })
})

In [7]:
#It create a mapping relationship betweeen label and its index
labels = [label for label in dataset['train'].features.keys() if label not in ['ID', 'Tweet']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['Optimistic',
 'Thankful',
 'Empathetic',
 'Pessimistic',
 'Anxious',
 'Sad',
 'Annoyed',
 'Denial',
 'Official report',
 'Surprise',
 'Joking']

In [41]:
# Read the articles file into a DataFrame
file_path = '/content/drive/MyDrive/Sentiment Analysis/guardian_articles.csv'
df = pd.read_csv(file_path)

In [9]:
df.shape

(149839, 7)

In [10]:
# Define the place to save the trained model
output_model_dir = '/content/drive/My Drive/Sentiment Analysis/Fine Tuned BERT Model'

In [11]:
# Load the trained BERT model
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(output_model_dir)

In [12]:
# Define the sample data
text = df['bodyContent'][0]

In [13]:
# Define the tokenization
tokens = tokenizer(text, add_special_tokens = False, truncation = False, return_tensors="pt")

Token indices sequence length is longer than the specified maximum sequence length for this model (1324 > 512). Running this sequence through the model will result in indexing errors


In [17]:
# Tokenizes the text with truncation to maximal_text_length and without special tokens.
def tokenize_text_with_truncation(
    text: str, tokenizer: PreTrainedTokenizerBase, maximal_text_length: int
) -> BatchEncoding:
    tokens = tokenizer(
        text, add_special_tokens=False, max_length=maximal_text_length, truncation=True, return_tensors="pt"
    )
    return tokens

In [18]:
# Tokenizes the text with truncation to maximal_text_length and without special tokens.
def tokenize_whole_text(text: str, tokenizer: PreTrainedTokenizerBase) -> BatchEncoding:
    tokens = tokenizer(text, add_special_tokens=False, truncation=False, return_tensors="pt")
    return tokens


In [19]:
# Adds special CLS token (token id = 101) at the beginning.
# Adds SEP token (token id = 102) at the end of each chunk.#
# Adds corresponding attention masks equal to 1 (attention mask is boolean).

def add_special_tokens_at_beginning_and_end(input_id_chunks: list[Tensor], mask_chunks: list[Tensor]) -> None:
    for i in range(len(input_id_chunks)):
        # adding CLS (token id 101) and SEP (token id 102) tokens
        input_id_chunks[i] = torch.cat([Tensor([101]), input_id_chunks[i], Tensor([102])])
        # adding attention masks corresponding to special tokens
        mask_chunks[i] = torch.cat([Tensor([1]), mask_chunks[i], Tensor([1])])

In [20]:
# Adds padding tokens (token id = 0) at the end to make sure that all chunks have exactly 512 tokens.
def add_padding_tokens(input_id_chunks: list[Tensor], mask_chunks: list[Tensor]) -> None:
    for i in range(len(input_id_chunks)):
        # get required padding length
        pad_len = 512 - input_id_chunks[i].shape[0]
        # check if tensor length satisfies required chunk size
        if pad_len > 0:
            # if padding length is more than 0, we must add padding
            input_id_chunks[i] = torch.cat([input_id_chunks[i], Tensor([0] * pad_len)])
            mask_chunks[i] = torch.cat([mask_chunks[i], Tensor([0] * pad_len)])

In [21]:
# Reshapes data to a form compatible with BERT model input.
def stack_tokens_from_all_chunks(input_id_chunks: list[Tensor], mask_chunks: list[Tensor]) -> tuple[Tensor, Tensor]:
    input_ids = torch.stack(input_id_chunks)
    attention_mask = torch.stack(mask_chunks)

    return input_ids.long(), attention_mask.int()

In [22]:
# Helper function for dividing 1-dimensional tensors into overlapping chunks.
def split_overlapping(tensor: Tensor, chunk_size: int, stride: int, minimal_chunk_length: int) -> list[Tensor]:
    check_split_parameters_consistency(chunk_size, stride, minimal_chunk_length)
    result = [tensor[i : i + chunk_size] for i in range(0, len(tensor), stride)]
    if len(result) > 1:
        # ignore chunks with less than minimal_length number of tokens
        result = [x for x in result if len(x) >= minimal_chunk_length]
    return result

In [23]:
# Split the tokens into the smaller chunks
def split_tokens_into_smaller_chunks(
    tokens: BatchEncoding,
    chunk_size: int,
    stride: int,
    minimal_chunk_length: int,
) -> tuple[list[Tensor], list[Tensor]]:
    input_id_chunks = split_overlapping(tokens["input_ids"][0], chunk_size, stride, minimal_chunk_length)
    mask_chunks = split_overlapping(tokens["attention_mask"][0], chunk_size, stride, minimal_chunk_length)
    return input_id_chunks, mask_chunks


In [24]:
# Check if all the chuck size are under 510
def check_split_parameters_consistency(chunk_size: int, stride: int, minimal_chunk_length: int) -> None:
    if chunk_size > 510:
        raise InconsistentSplittingParamsException("Size of each chunk cannot be bigger than 510!")
    if minimal_chunk_length > chunk_size:
        raise InconsistentSplittingParamsException("Minimal length cannot be bigger than size!")
    if stride > chunk_size:
        raise InconsistentSplittingParamsException(
            "Stride cannot be bigger than size! Chunks must overlap or be near each other!"
        )

In [25]:
# Warp all the steps into a single function
def transform_single_text(
    text: str,
    tokenizer: PreTrainedTokenizerBase,
    chunk_size: int,
    stride: int,
    minimal_chunk_length: int,
    maximal_text_length: Optional[int],
    ) -> tuple[Tensor, Tensor]:
    if maximal_text_length:
      tokens = tokenize_text_with_truncation(text, tokenizer, maximal_text_length)
    else:
      tokens = tokenize_whole_text(text, tokenizer)
    input_id_chunks, mask_chunks = split_tokens_into_smaller_chunks(tokens, chunk_size, stride, minimal_chunk_length)
    add_special_tokens_at_beginning_and_end(input_id_chunks, mask_chunks)
    add_padding_tokens(input_id_chunks, mask_chunks)
    input_ids, attention_mask = stack_tokens_from_all_chunks(input_id_chunks, mask_chunks)
    return input_ids, attention_mask

In [26]:
# Get the results, inputs id is encoding the words as integers, attention masks is the position of the padded indices
input_ids, attention_mask = transform_single_text(text, tokenizer, 510, 510, 1, None)

In [27]:
# We can get the output with this two inputs, which will be the matrix of the results
model_output = model(input_ids, attention_mask)

In [28]:
#The logits is a tensor that contains the scores for every individual label.
logits = model_output.logits
logits.shape

torch.Size([3, 10])

In [29]:
# apply sigmoid + threshold to get the results for each part of essay
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.cpu())
predictions = np.zeros(probs.shape)
for i in range(len(predictions)):  # Iterate over each part of the essay
    predictions_part = np.zeros(probs[i].shape)
    predictions_part[np.where(probs[i] >= 0.5)] = 1
    # turn predicted id's into actual label names for this part
    predicted_labels_part = [id2label[idx] for idx, label in enumerate(predictions_part) if label == 1.0]
    print("Part", i+1, "predicted labels:", predicted_labels_part)

Part 1 predicted labels: ['Annoyed']
Part 2 predicted labels: ['Official report']
Part 3 predicted labels: ['Official report']


In [32]:
# We combine the results together for the different part of essays
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.cpu())
all_predicted_labels = []

for i in range(len(probs)):  # Iterate over each part of the essay
    predictions_part = np.zeros(probs[i].shape)
    predictions_part[np.where(probs[i] >= 0.5)] = 1
    # turn predicted id's into actual label names for this part
    predicted_labels_part = [id2label[idx] for idx, label in enumerate(predictions_part) if label == 1.0]
    all_predicted_labels.extend(predicted_labels_part)

# Remove duplicates and return unique predicted labels
unique_predicted_labels = list(set(all_predicted_labels))
print("Unique predicted labels:", unique_predicted_labels)


Unique predicted labels: ['Annoyed', 'Official report']


In [42]:
# Remove the body content data, which is not string
df = df[df["bodyContent"].apply(lambda x: isinstance(x, str))]

In [ ]:
# Process each text with the all the functions and create a new column to store the results
import pandas as pd

# Function to process a single text
def process_single_text(text):
    input_ids, attention_mask = transform_single_text(text, tokenizer, 510, 510, 1, None)
    model_output = model(input_ids, attention_mask)
    logits = model_output.logits
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.cpu())
    all_predicted_labels = []

    for i in range(len(probs)):  # Iterate over each part
        predictions_part = np.zeros(probs[i].shape)
        predictions_part[np.where(probs[i] >= 0.5)] = 1
        predicted_labels_part = [id2label[idx] for idx, label in enumerate(predictions_part) if label == 1.0]
        all_predicted_labels.extend(predicted_labels_part)

    # Remove duplicates and return unique predicted labels
    unique_predicted_labels = list(set(all_predicted_labels))
    return unique_predicted_labels

# Process 1000 texts from the DataFrame and store results in a new column
texts_to_process = df["bodyContent"].head(1000)
df["labels"] = texts_to_process.apply(process_single_text)
